# Web scrapping for Data Scientist job in CO(Total points 9)

In this exercise we'll do web scrapping for **Data Scientist job in CO**


Here is the link to the search query

https://www.indeed.com/jobs?q=data+scientist&l=CO

As you can see at the bottom of the page there are link to series of pages related to this search.
If you click on second page, search url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO&start=10

If you click on 3rd then url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO&start=20

Hence to go to more pages we can format search string(**change start=??** part) for **requests.get in a loop**


# Q1(5 =  4(non indicator columns) + 1(indicator columns) points) Please complete the following task

- Scrape 10 pages (**last page(10 th) url will be https://www.indeed.com/jobs?q=data+scientist&l=CO&start=90**)and build a pandas DataFrame containing following information
    + **job title, name of the company, location, summary of job description**
    + **Indicator columns(with value True/False) about keywords Python, SQL, AWS, RESTFUL, Machine learning, Deep Learning, Text Mining, NLP, SAS, Tableau, Sagemaker, TensorFlow, Spark**

Note:
- Make sure that you do a case insensitive search for keywords when filing(Tue/False) in the indicator columns
- You need to go to the webpage of detail job posting for keywords search. Main job posting only contains summary of the job description.  Build detail job posting webpage url  from web scrapping main search results.

In [200]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

keywords = ["Python", "SQL", "AWS", "RESTFUL", "Machine learning", "Deep Learning", "Text Mining", "NLP", "SAS", "Tableau", "Sagemaker", "TensorFlow", "Spark", "Clearance"]


frame_data = []

for i in range(0, 10):
    if i == 0:
        url = 'https://www.indeed.com/jobs?q=data+scientist&l=CO'
    else:
        url = 'https://www.indeed.com/jobs?q=data+scientist&l=CO&start='+str(i * 10)
       
    # print(url)
    
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'html.parser')
    job_list = soup.find_all("div", {"class": "jobsearch-SerpJobCard"})
    
    # print(len(job_list))
    for item in job_list:
        title_list = item.select('h2.title a')
        data_row = {}
        for row in title_list:
            href = row['href']
            title = row['title']

            data_row['title'] = title
            

            # print(title)
            url1 = href.replace("/rc/clk", "https://www.indeed.com/viewjob")
            # print(url1)
#             url1 = "https://www.indeed.com/viewjob?jk=8a5b0137fa9abb46&from=serp&vjs=3"

            try: 
                page1 = requests.get(url1)

                soup1 = BeautifulSoup(page1.content, 'html.parser')
        #         title_div = soup1.find("div", {"class": "jobsearch-JobInfoHeader-title-container"})
        #         title = title_div.find("h1")
        #         print(title)    
                desc_div = soup1.find("div", {"id": "jobDescriptionText"})
                desc = desc_div.text 

                data_row['desc'] = desc

                desc1 = desc.lower()

                for k in keywords:               
                    data_row[k] = k.lower() in desc1

            except:
                data_row['desc'] = ""
                for k in keywords:               
                    data_row[k] = False



            break

        company_list = item.select('span.company a')
        if len(company_list) > 0:
            company = company_list[0].text
            company = company.strip("\n")
            # print(company)
            data_row['company'] = company


        location_list = item.select('span.location')
        if len(location_list) > 0:
            location = location_list[0].text
            # print(location)
            data_row['location'] = location

        salary_list = item.select('span.salaryText')
        if len(salary_list) > 0:
            salary = salary_list[0].text
            # print(location)
            data_row['salary'] = salary

        frame_data.append(data_row)

#         break

    

df = pd.DataFrame(frame_data)
df

In [192]:
# df['Clearance']

# Q2(1 point) Save you DataFrame to pickle file name *indeed_job_co.pkl*. 
   Load this pkl file in dataFrame and use this dataFrame for answering following questions.

   <font color='red'>upload the pickle file(indeed_job_co.pkl) along with solution notebook to the canvas</font>

In [193]:
#write code here
# df.to_pickle("indeed_job_co.pkl")
# df.to_csv("indeed_job_co.csv")

df = pd.read_pickle('indeed_job_co.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             155 non-null    object
 1   desc              155 non-null    object
 2   Python            155 non-null    bool  
 3   SQL               155 non-null    bool  
 4   AWS               155 non-null    bool  
 5   RESTFUL           155 non-null    bool  
 6   Machine learning  155 non-null    bool  
 7   Deep Learning     155 non-null    bool  
 8   Text Mining       155 non-null    bool  
 9   NLP               155 non-null    bool  
 10  SAS               155 non-null    bool  
 11  Tableau           155 non-null    bool  
 12  Sagemaker         155 non-null    bool  
 13  TensorFlow        155 non-null    bool  
 14  Spark             155 non-null    bool  
 15  Clearance         155 non-null    bool  
 16  company           123 non-null    object
 17  salary          

<font size = "6" color='red'> Use pandas functionality to answer question 2</font>
# Q 3 a(1 point) Which city has maximum job posting.



df1 = df['location'].value_counts().rename_axis('location').reset_index(name='counts').sort_values(by=['counts'])
print(df1.loc[0, 'location'], ",", df1.loc[0, 'counts'])


In [194]:
df['salary'] = df['salary'].replace('\n', '', regex=True)
df['salary'] = df['salary'].replace(' a year', '', regex=True)
# df['salary'] = df['salary'].replace('From', '', regex=True)
# df['salary'] = df['salary'].replace('35 an hour', '', regex=True)
# df['salary'] = df['salary'].replace('', '', regex=True)




df[['salary_min', 'salary_max']] = df['salary'].str.split('-', expand=True)
df['salary_min'] = df['salary_min'].str.replace('$', '').str.replace(',', '').str.replace('From ', '').str.replace(' an hour', '')
df['salary_max'] = df['salary_max'].str.replace('$', '').str.replace(',', '').str.replace(' an hour', '')
# df['salary_min'] = df['salary_min'].astype(int)
# df['salary_max'] = df['salary_max'].astype(int)


# df[['Avg Salary']] = (df['salary_min'] + df['salary_max'])

cols = ["title", "company", "desc", "location", "salary_min", "salary_max", "Clearance", "Python", "SQL", "AWS",
        "RESTFUL", "Machine learning", "Deep Learning", "Text Mining", "NLP", "SAS", "Tableau", "Sagemaker",
        "TensorFlow", "Spark"]

df = df[cols]
df


,title,company,desc,location,salary_min,salary_max,Clearance,Python,SQL,AWS,RESTFUL,Machine learning,Deep Learning,Text Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark
0,Data Scientist,Arrow Electronics,,NaN,NaN,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Artificial Intelligence/Machine Learning Data ...,Mitre Corporation,,NaN,NaN,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,"Sr. Healthcare Data Analyst (Remote, USA)",NaN,,NaN,70000,90000,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,AI Chatbot Engineer,Verizon,,NaN,107200,199200,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,"Senior Consultant - Advanced Solutions, Energy...",Guidehouse,,NaN,72000,80000,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Data Scientist,CyberCoders,,NaN,95000,110000,False,False,False,False,False,False,False,False,False,False,False,False,False,False
151,Cloud Data Engineer - Data Scientist (Remote),Sealaska Corporation,Overview:\n\nCognitell is seeking a Data Engin...,"Denver, CO",NaN,None,False,True,True,True,False,True,True,False,False,False,False,True,True,True
152,Statistical Programmer II,PAREXEL,,NaN,NaN,None,False,False,False,False,False,False,False,False,False,False,False,False,False,False
153,Machine Learning/A.I. Engineer,NaN,Overview:\nAt Josh.ai we are innovating at the...,"Denver, CO",NaN,None,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [210]:
df2 = df
df2['salary_min'].astype(np.float).astype("Int32")
df2['salary_max'].astype(np.float).astype("Int32")
df2['Avg Salary'] = df2['salary_min'] + df2['salary_max']
df2

,title,company,desc,location,salary_min,salary_max,Clearance,Python,SQL,AWS,...,Machine learning,Deep Learning,Text Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark,Avg Salary
0,Data Scientist,Arrow Electronics,,NaN,NaN,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NaN
1,Artificial Intelligence/Machine Learning Data ...,Mitre Corporation,,NaN,NaN,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NaN
2,"Sr. Healthcare Data Analyst (Remote, USA)",NaN,,NaN,70000,90000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,70000 90000
3,AI Chatbot Engineer,Verizon,,NaN,107200,199200,False,False,False,False,...,False,False,False,False,False,False,False,False,False,107200 199200
4,"Senior Consultant - Advanced Solutions, Energy...",Guidehouse,,NaN,72000,80000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,72000 80000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Data Scientist,CyberCoders,,NaN,95000,110000,False,False,False,False,...,False,False,False,False,False,False,False,False,False,95000 110000
151,Cloud Data Engineer - Data Scientist (Remote),Sealaska Corporation,Overview:\n\nCognitell is seeking a Data Engin...,"Denver, CO",NaN,None,False,True,True,True,...,True,True,False,False,False,False,True,True,True,NaN
152,Statistical Programmer II,PAREXEL,,NaN,NaN,None,False,False,False,False,...,False,False,False,False,False,False,False,False,False,NaN
153,Machine Learning/A.I. Engineer,NaN,Overview:\nAt Josh.ai we are innovating at the...,"Denver, CO",NaN,None,False,False,False,False,...,True,False,False,False,False,False,False,False,False,NaN


# Q 3 b(1.5 point) - Top 3 most demanding skills(like Python, AWS, SQL ...)



histogram = [0] * len(keywords)
for index, row in df.iterrows():        
    j = 0
    for k in keywords:
        if row[k] == True:
            histogram[j] += 1
            
        j += 1
            
print(histogram)

Z = [x for _, x in sorted(zip(histogram, keywords), reverse=True)]

# print(Z[0:3])

print(Z)

# Q3 c(.5 point) What other questions you would like to ask  based on indeed data?
# Top skills required for artifical intelligence and machine learning jobs
# Highest paid skills
# Highest paying job titles by company type
This is free response questions.

Question -

Average pay for jobs requiring security clearance and average pay for non-security clearance jobs?

*Write your anser in this cell. Double click this cell to edit this cell*
1. Python, NLP, TensorFlow, Spark
2. Python, AWS, Machine learning, NLP, SAS